In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 9.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660424 sha256=82327545ac970278b7b6816699488cdef40fb045be19ee6260910ea841dd086d
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [ ]:
import numpy as np
import pycuda.autoinit # Initializes CUDA context
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import time

# Embed the CUDA kernel code as a Python string
cuda_source_code = """
// rgb_to_grayscale_kernel.cu
#include <cuda_runtime.h> // Includes vector_types.h (for float3) and math.h (for fmaf)

// Optimized kernel using float3 for memory access
__global__ void rgb_to_grayscale_float3(const float *__restrict__ input_raw,
                                      float *__restrict__ output,
                                      int width, int height) {
    // Calculate global thread coordinates
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    // Boundary check
    if (x < width && y < height) {
        int pixel_idx = y * width + x; // 1D index for the current pixel

        const float3* input_float3_ptr = (const float3*)input_raw;

        float3 rgb_val = input_float3_ptr[pixel_idx];

        float gray_val = 0.2989f * rgb_val.x; // R component
        gray_val = fmaf(0.5870f, rgb_val.y, gray_val); // Add G component term
        gray_val = fmaf(0.1140f, rgb_val.z, gray_val); // Add B component term

        output[pixel_idx] = gray_val;
    }
}


// Bonus: Optimized kernel using shared memory.
#define TILE_DIM_X 16 // Must match blockDim.x used in host code for this kernel
#define TILE_DIM_Y 16 // Must match blockDim.y used in host code for this kernel

__global__ void rgb_to_grayscale_shared_mem(const float *__restrict__ input_raw,
                                          float *__restrict__ output,
                                          int width, int height) {
    __shared__ float3 tile_s[TILE_DIM_Y][TILE_DIM_X];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int global_x = blockIdx.x * TILE_DIM_X + tx;
    int global_y = blockIdx.y * TILE_DIM_Y + ty;

    const float3* input_float3_ptr = (const float3*)input_raw;

    if (global_x < width && global_y < height) {
        tile_s[ty][tx] = input_float3_ptr[global_y * width + global_x];
    }

    __syncthreads();

    if (global_x < width && global_y < height) {
        float3 rgb_val = tile_s[ty][tx];

        float gray_val = 0.2989f * rgb_val.x;
        gray_val = fmaf(0.5870f, rgb_val.y, gray_val);
        gray_val = fmaf(0.1140f, rgb_val.z, gray_val);

        output[global_y * width + global_x] = gray_val;
    }
}

// Reference kernel (similar to problem statement)
__global__ void rgb_to_grayscale_reference(const float *__restrict__ input,
                                         float *__restrict__ output,
                                         int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int idx = y * width + x;
        int rgb_idx = idx * 3;

        float r = input[rgb_idx + 0];
        float g = input[rgb_idx + 1];
        float b = input[rgb_idx + 2];

        output[idx] = 0.2989f * r + 0.5870f * g + 0.1140f * b;
    }
}
"""

# Compile the CUDA source code
# Adding -Xptxas -v can give insights into resource usage (registers, shared memory)
module = SourceModule(cuda_source_code, options=['-Xptxas=-v']) #for verbose PTXAS output

# Get kernel functions from the compiled module
kernel_ref = module.get_function("rgb_to_grayscale_reference")
kernel_float3 = module.get_function("rgb_to_grayscale_float3")
kernel_shared_mem = module.get_function("rgb_to_grayscale_shared_mem")

print("CUDA Kernels Compiled Successfully!")

CUDA Kernels Compiled Successfully!


<ipython-input-2-469817ab6dfe>:93: UserWarning: The CUDA compiler succeeded, but said the following:
ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function 'rgb_to_grayscale_reference' for 'sm_75'
ptxas info    : Function properties for rgb_to_grayscale_reference
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 376 bytes cmem[0]
ptxas info    : Compiling entry function 'rgb_to_grayscale_shared_mem' for 'sm_75'
ptxas info    : Function properties for rgb_to_grayscale_shared_mem
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 14 registers, 3072 bytes smem, 376 bytes cmem[0]
ptxas info    : Compiling entry function 'rgb_to_grayscale_float3' for 'sm_75'
ptxas info    : Function properties for rgb_to_grayscale_float3
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 376 bytes cmem[0]

  module = SourceModule(cuda_source_code, options

In [3]:
def test_kernel(kernel_func, kernel_name, width, height, input_rgb_np):
    """Tests a given CUDA kernel for correctness and performance."""
    print(f"\n--- Testing {kernel_name} for {width}x{height} image ---")

    # Prepare host output array
    output_gray_np = np.zeros((height, width), dtype=np.float32)

    # Allocate GPU memory
    input_gpu = cuda.mem_alloc(input_rgb_np.nbytes)
    output_gpu = cuda.mem_alloc(output_gray_np.nbytes)

    # Copy input data from host (CPU) to device (GPU)
    cuda.memcpy_htod(input_gpu, input_rgb_np)

    # Define CUDA kernel launch parameters (block and grid dimensions)
    if kernel_name == "Optimized Kernel (Shared Memory)":
        block_dim_x, block_dim_y = 16, 16 # Must match #define TILE_DIM_X/Y
    else:
        block_dim_x, block_dim_y = 16, 16 # Common default, 256 threads/block

    block_dims = (block_dim_x, block_dim_y, 1)
    grid_dims = ((width + block_dims[0] - 1) // block_dims[0],
                 (height + block_dims[1] - 1) // block_dims[1],
                 1)

    # Warm-up run (important for stable performance measurements)
    kernel_func(input_gpu, output_gpu, np.int32(width), np.int32(height),
                block=block_dims, grid=grid_dims)
    cuda.Context.synchronize() # Ensure warm-up is complete

    # Measure execution time using CUDA events for precision
    num_runs = 100 # Average over multiple runs for stability
    start_event = cuda.Event()
    end_event = cuda.Event()

    start_event.record() # Record start time
    for _ in range(num_runs):
        kernel_func(input_gpu, output_gpu, np.int32(width), np.int32(height),
                    block=block_dims, grid=grid_dims)
    end_event.record() # Record end time

    end_event.synchronize() # Wait for all kernel executions to complete

    elapsed_ms_total = start_event.time_till(end_event) # Total time for num_runs in ms
    elapsed_ms_per_run = elapsed_ms_total / num_runs

    print(f"Execution time: {elapsed_ms_per_run:.4f} ms (averaged over {num_runs} runs)")

    # Copy output data from device (GPU) to host (CPU)
    cuda.memcpy_dtoh(output_gray_np, output_gpu)

    # Verify correctness against NumPy calculation
    ref_gray_np = (0.2989 * input_rgb_np[:, :, 0] +
                   0.5870 * input_rgb_np[:, :, 1] +
                   0.1140 * input_rgb_np[:, :, 2])

    try:
        np.testing.assert_almost_equal(output_gray_np, ref_gray_np, decimal=4)
        print("Correctness: PASS")
    except AssertionError as e:
        print("Correctness: FAIL")
        print(e)

    # Free GPU memory
    input_gpu.free()
    output_gpu.free()

    return elapsed_ms_per_run


if __name__ == "__main__":
    # Define image sizes to test (must be square and have even dimensions)
    image_sizes = [(512, 512), (1024, 1024), (2048, 2048)]

    results_summary = {}

    for width, height in image_sizes:
        if not (width % 2 == 0 and height % 2 == 0 and width == height):
            print(f"Skipping size {width}x{height} as it does not meet constraints.")
            continue

        print(f"\n======================================================")
        print(f"Processing Image Size: {width}x{height}")
        print(f"======================================================")

        input_rgb_np = np.random.rand(height, width, 3).astype(np.float32)

        kernels_to_test = [
            (kernel_ref, "Reference Kernel"),
            (kernel_float3, "Optimized Kernel (float3)"),
            (kernel_shared_mem, "Optimized Kernel (Shared Memory)")
        ]

        size_str = f"{width}x{height}"
        for kernel_func, kernel_name in kernels_to_test:
            if kernel_name not in results_summary:
                results_summary[kernel_name] = {}

            exec_time = test_kernel(kernel_func, kernel_name, width, height, input_rgb_np)
            results_summary[kernel_name][size_str] = exec_time

    # Print summary table
    print("\n\n--- Performance Summary (ms) ---")
    header_cols = [f"{w}x{h}" for w,h in image_sizes if w%2==0 and h%2==0 and w==h]
    header = f"| {'Kernel Name':<30} |" + "".join([f" {col.center(12)} |" for col in header_cols])
    separator = "-" * len(header)

    print(separator)
    print(header)
    print(separator)

    # Ensure a consistent order for printing results
    kernel_order = ["Reference Kernel", "Optimized Kernel (float3)", "Optimized Kernel (Shared Memory)"]
    for kernel_name_key in kernel_order:
        if kernel_name_key in results_summary:
            row = f"| {kernel_name_key:<30} |"
            for size_key in header_cols:
                time_val = results_summary[kernel_name_key].get(size_key, "N/A")
                if isinstance(time_val, float):
                    row += f" {time_val:>10.4f}ms |"
                else:
                    row += f" {'N/A'.center(12)} |"
            print(row)
    print(separator)


Processing Image Size: 512x512

--- Testing Reference Kernel for 512x512 image ---
Execution time: 0.0185 ms (averaged over 100 runs)
Correctness: PASS

--- Testing Optimized Kernel (float3) for 512x512 image ---
Execution time: 0.0186 ms (averaged over 100 runs)
Correctness: PASS

--- Testing Optimized Kernel (Shared Memory) for 512x512 image ---
Execution time: 0.0208 ms (averaged over 100 runs)
Correctness: PASS

Processing Image Size: 1024x1024

--- Testing Reference Kernel for 1024x1024 image ---
Execution time: 0.0694 ms (averaged over 100 runs)
Correctness: PASS

--- Testing Optimized Kernel (float3) for 1024x1024 image ---
Execution time: 0.0693 ms (averaged over 100 runs)
Correctness: PASS

--- Testing Optimized Kernel (Shared Memory) for 1024x1024 image ---
Execution time: 0.0716 ms (averaged over 100 runs)
Correctness: PASS

Processing Image Size: 2048x2048

--- Testing Reference Kernel for 2048x2048 image ---
Execution time: 0.2639 ms (averaged over 100 runs)
Correctness: 